# Accessing and creating content

Your GIS can host a [variety of maps, web layers, analytical tools, apps](http://doc.arcgis.com/en/arcgis-online/reference/geo-info.htm) and [individual files](http://doc.arcgis.com/en/arcgis-online/share-maps/supported-items.htm). Using the `gis` module, you can search for, access and manage all your folders, contents and automate such maintenance workflows as scripts.

## What is Content?

- Digital information
- Excel spreadsheets, spatial data, etc..
- Services
- Web Maps
- Imagery and Raster Data

## What a User Sees

<img src="./img/sample_item.png" />

## What a Programmer Sees


In [ ]:
from arcgis.gis import GIS
gis = GIS(profile='your_online_profile')
item = gis.content.get("ab399b847323487dba26809bf11ea91a")
print(item)

In [ ]:
item.__dict__

## Import libraries

In [ ]:
import os
import datetime as dt
import pandas 
from arcgis.gis import GIS

## About search

As mentioned in the [`gis module`](https://developers.arcgis.com/python/guide/the-gis-module/) guide, the Python API uses [Resource Manager classes](https://developers.arcgis.com/python/guide/the-gis-module/) to manage Web GIS users, groups, datastores, and content. You access the [`UserManager`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.UserManager), [`GroupManager`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#groupmanager) and [`ContentManager`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager) classes as the `users`, `groups`, and `content` properties of the [`GIS`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis), respectively. (See [`Helper objects`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis) for details.)

Each resource manager implements the [ArcGIS REST API](https://developers.arcgis.com/rest/users-groups-and-items/search.htm) `search` operation as a method. For example, `gis.content.search()`. It's important to understand that the `search` mechanism uses many different inputs to find possible matches, ranks them and returns appropriate results. The `search` becomes ideal for human interaction, but `fuzzy` when looking for specific records programmatically. The search results are non-deterministic. Using `search` may not necessarily be the best approach for finding specific items, but more a group of items from which to further filter. 

It's also important to know that using `search` programmatically, like with the Python API, does not correspond identically to searching with an application written in a different language.  The various `search` options in the ArcGIS Online or Portal for ArcGIS Enterprise interfaces work differently than the Python API resource managers' `search`. Different applications may use different inputs. The relationship between a content `search` in one application, like a Portal or ArcGIS Online viewer, is not one-to-one with a content search using the Python API even when logged in as the same user.

Let's look at an example to clarify:

In [ ]:
# Search for Feature Layers owned by the logged-in user
my_content = gis.content.search(query="owner:" + gis.users.me.username, 
                                item_type="Feature Layer", 
                                max_items=15)

my_content

Lets see what you get when you access `content` property

## Creating new content

To create new items on your GIS, you use the [`add()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.add) method on a `ContentManager` instance. Again, rather than creating an instance of the `ContentManager` directly, the Python API implments the class as the `content` property of your `GIS` object.

The `add()` method accepts a dictionary containing the properties of the item to be created. The optional `data` parameter accepts a string path to a file. You can use the optional `metadata` parameter to specify an XML file containing metadata information.

All content you add this way is added to the authenticated user's contents. Let us connect to our Enterprise instance, add a csv file as an item and then publish a feature layer from it.

For example, set up a path to the data:
`data_path = os.path.join(r"<your_drive>:\\", "path", "to", "data")`

In [ ]:
data_path = (r".\data")

In [ ]:
csv_path = os.path.join(data_path, "earthquakes2.csv")
csv_properties={'title':'Earthquakes around the world from 1800s to early 1900s',
                'description':'Measurements from globally distributed seismometers',
                'tags':'arcgis, python, earthquake, natural disaster, emergency'}
thumbnail_path = os.path.join(data_path, "earthquake.png")

earthquake_csv_item = gis.content.add(item_properties=csv_properties, data=csv_path,
                                     thumbnail = thumbnail_path)

The `add()` method returns an object of type `arcgis.gis.Item` representing the added item. In Jupyter notebook environment, you can visualize it by querying the item

In [ ]:
earthquake_csv_item

**Note**: You can specify most properties of an item as a dictionary to the `item_properties` parameter. Refer to the [API ref doc](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.add) of this method for a definitive list. You could also specify an empty dictionary to this parameter, the downside is search cannot index these items efficiently if it does not have sufficient metadata information, hence that is not a good programming practice.

### Publishing an item as a web layer
In the example above you added a csv [`Item`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#item) to the portal. Items such as csv files, service definitions, file geodatabases, shapefiles and packages such as tile, scene and vector tile packages can be published into hosted web layers. You call the [`publish()`](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.Item.publish) method on the `Item` object to create the web layer.

Optionally, The `publish()` method accepts an optional `publish_parameters` dictionary where you can specify certain parameters. See the [publish REST API](http://resources.arcgis.com/en/help/arcgis-rest-api/index.html#/Publish_Item/02r300000080000000/) documentation for detailed descriptions on these parameters. For instance, the `address_fields` parameter allows you to specify which field in your csv or dataset contains address fields suitable for geocoding.

Let us publish the csv into a web feature layer. This csv contains latitude and longitude coordinates of each point, hence the `address_fields` parameter can be ignored. The `publish()` method returns an `Item` object corresponding to the web layer created.

In [ ]:
earthquake_feature_layer_item = earthquake_csv_item.publish()

In [ ]:
earthquake_feature_layer_item

Notice that the GIS used the same metadata (thumbnail, title, description) of the source item used for publishing. This helps provide context on what the new item is and also as a good starting point for the metadata which you can edit later.

<a id="importing-data-from-a-pandas-data-frame"></a>
### Importing data from a pandas data frame

[Pandas](http://pandas.pydata.org) is a popular data analysis library for Python. The data frame object of pandas allows you to store and analyze tabular information with ease.

Let us create a new pandas data frame representing locations of a few coastal cities read from a csv file. Note, in the example below, you could publish the csv as an item as shown earlier, but we will observe how you can filter that data through pandas and create a Spatially Enabled dataframe instead

In [ ]:
# read csv as a pandas dataframe
ports_df = pandas.read_csv(os.path.join(data_path, "ports.csv"))
ports_df

In [ ]:
# find latitude of SFO
lat = ports_df.loc[ports_df.port_name == 'SAN FRANCISCO']['latitude']
lat

In [ ]:
# only select ports that are to the south of SFO
ports_south_of_SFO = ports_df.loc[ports_df.latitude < lat[0]]
ports_south_of_SFO

Now that we have the desired data frame, let us transform it to a SeDF and publish it.

In [ ]:
sedf = pandas.DataFrame.spatial.from_xy(ports_south_of_SFO,"longitude","latitude")
sedf

In [ ]:
sedf.spatial.plot()

In [ ]:
sedf.spatial.to_featurelayer("Ports south of SFO")

<a id="organizing-content"></a>
## Organizing content
Over time, your GIS can get filled with content. You can create folders on your GIS, just like in your file system, and use it to organize.

<a id="using-folders-to-organize-your-content"></a>
### Using folders to organize your content
The `create_folder()` and `delete_folder()` methods available on the `ContentManager` class can be used to manage your folders. Once created, you can use the `move()` method to move content into and out of folders.

Let us create a folder called 'ports' and move the `ports_item` into it 

In [ ]:
# create new folder
gis.content.create_folder(folder='ports2')

In [ ]:
# move the ports_item into this folder
ports_item.move(folder='ports')

You can move content out of folders to root by calling the `move()` method and specifying `/` as folder name

In [ ]:
# move back to root
ports_item.move(folder='/')

## Summary
Thus, in this guide you observed how the `gis` module can be used to create new content and search for existing ones. Head over to the guide on [managing-your-content](https://developers.arcgis.com/python/guide/managing-your-content) next. When combined into a script, the ArcGIS API for Python becomes a powerful tool to automate your GIS content creation and management tasks.